In [61]:
import seaborn as sn
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import SnowballStemmer
from gensim.models import Word2Vec

train = pd.read_csv('/Users/kirill/Downloads/all_auto_ru_09_09_2020.csv') # датасет для обучения модели
test = pd.read_csv('/Users/kirill/Downloads/sf-dst-car-price-prediction/test.csv')

In [62]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', 100)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)

In [63]:
#проводим предварительный анализ, на основе этих данных выполняю дальнейшие преобразования.
#from pandas_profiling import ProfileReport
#ProfileReport(data)
#corr_matrix = test.corr()
#print(corr_matrix)

#sn.heatmap(corr_matrix, annot=True)

#удаляем productionDate и engineDisplacement: высокая корр.

In [64]:
#обработка пропусков
test['ПТС'][test['ПТС'].isnull()] = 'Оригинал'
train.dropna(subset=['price', 'bodyType', 'description'], inplace = True)
train['ПТС'].fillna('ORIGINAL', inplace=True)
train['Владельцы'].fillna(3, inplace=True)


/var/folders/50/76rnxkws18g9dgds3nmk0gvh0000gn/T/ipykernel_1684/2556644217.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['ПТС'][test['ПТС'].isnull()] = 'Оригинал'


In [65]:
#в этой ячейке приводим данные в нужный нам вид
#в тестовой выборке оставляем:
test_col = ['bodyType', 'brand', 'color', 'fuelType', 'mileage', 'model_name', 'numberOfDoors', 
     'productionDate', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'enginePower', 'description']
#потом можно что-то подумать с description и Владение(много пропусков)

train_col = ['bodyType', 'brand', 'color', 'fuelType', 'mileage', 'model', 'numberOfDoors',
             'productionDate', 'vehicleTransmission', 'Владельцы', 'ПТС', 'Привод', 'Руль', 'enginePower'
             , 'price', 'description']

train = train[train_col]
test = test[test_col]

train.rename(columns={'model': 'model_name'}, inplace=True)

test['price'] = 0

train.numberOfDoors = train.numberOfDoors.astype('Int64')
train.enginePower = train.enginePower.astype('Int64')
train.price = train.price.astype('Int64')
train['Владельцы'] = train['Владельцы'].astype('Int64')

test['Владельцы'] = test['Владельцы'].apply(lambda s: s.replace(u'\xa0',' '))
test['Владельцы'] = test['Владельцы'].apply(lambda s: 3 if s=='3 или более' else 2 if s=='2 владельца' else 1)

test['enginePower'] = test['enginePower'].apply(lambda s: int(s.split(' ')[0]))

dict_color = {'040001': 'чёрный', 'FAFBFB': 'белый', 'CACECB': 'серебристый', '97948F': 'серый', '0000CC': 'синий','EE1D19': 'красный', '200204': 'коричневый',
                '007F00': 'зелёный', 'C49648': 'бежевый', '22A0F8': 'голубой', '660099': 'пурпурный', 'DEA522': 'золотистый',
              '4A2197': 'фиолетовый', 'FFD600': 'жёлтый', 'FF8649': 'оранжевый', 'FFC0CB': 'розовый'}
train.replace(dict_color, inplace=True)

train['vehicleTransmission'] = train['vehicleTransmission'].apply(lambda s: 'автоматическая' if s=='AUTOMATIC' 
                                                                      else 'механическая' if s=='MECHANICAL' 
                                                                      else 'роботизированная' if s=='ROBOT'
                                                                      else 'вариатор')

train['ПТС'] = train['ПТС'].apply(lambda s: 'Оригинал' if s=='ORIGINAL' else 'Дубликат')

train['Руль'] = train['Руль'].apply(lambda s: 'Левый' if s=='LEFT' else 'Правый')

#на всякий случай переводим всю строковую инфу в нижний регистр
for col in ['bodyType','brand','color','fuelType','model_name','vehicleTransmission','ПТС','Привод','Руль', 'description']:
    train[col] = train[col].apply(lambda s: s.lower())
    test[col] = test[col].apply(lambda s: s.lower())

In [66]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
df = pd.concat([train, test], ignore_index=True)

In [67]:
df.bodyType = df.bodyType.apply(lambda x: x.lower().split()[
                                0].strip() if isinstance(x, str) else x)

In [68]:
#формируем новые признаки на основе description
dealer_labels = set(['первоначальный взнос', 'trade-in', 'рассрочка'])
df['dealer'] = df.description.apply(lambda s: 1 if len(dealer_labels & set(s.split(' ')))>0 else 0)

df['gift'] = df.description.apply(lambda s: 1 if 'в подарок' in s else 0)
df['broken'] = df.description.apply(lambda s: 1 if 'не на ходу' in s else 0)



super_cond_labels = ['в отличном состоянии', 'в хорошем состоянии', 'состояние отличное', 'состояние хорошее',
                    'отличное состояние', 'хорошее состояние', 'в идеальном состоянии', 'состояние идеальное', 
                     'состояние идеальное', 'идеальное состояние', 'вложений не требует']
df['super_cond'] = 0
for i in range(len(df.description)):
    for label in super_cond_labels:
        if label in df.description[i]:
            df['super_cond'][i]=1
            break
            
df['bad_cond'] = 0
for i in range(len(df.description)):
    if 'требует ремонта' in df.description[i] and 'не требует ремонта' not in df.description[i]:
        df['bad_cond'][i]=1
    elif 'битая' in df.description[i] and 'не битая' not in df.description[i]:
        df['bad_cond'][i]=1
    elif 'битый' in df.description[i] and 'не битый' not in df.description[i]:
        df['bad_cond'][i]=1
            
df['torg'] = 0
for i in range(len(df.description)):
    if 'торг' in df.description[i]:
        df['torg'][i]=1
    elif 'торг уместен' in df.description[i]:
        df['torg'][i]=1 

df['no_torg'] = 0
for i in range(len(df.description)):
    if 'без торга' in df.description[i]:
        df['no_torg'][i]=1
    elif 'цена окончательная' in df.description[i]:
        df['no_torg'][i]=1



/var/folders/50/76rnxkws18g9dgds3nmk0gvh0000gn/T/ipykernel_1684/343431481.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['super_cond'][i]=1
/var/folders/50/76rnxkws18g9dgds3nmk0gvh0000gn/T/ipykernel_1684/343431481.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bad_cond'][i]=1
/var/folders/50/76rnxkws18g9dgds3nmk0gvh0000gn/T/ipykernel_1684/343431481.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['bad_cond'][i

In [69]:
#создаем дополнительные признаки для улучшения качества модели

df['descr_words_count'] = df['description'].apply(lambda s: len(s.split(' ')))
df['auto_age'] = df['productionDate'].apply(lambda x: int(2021-x))
#df['miles_per_year'] = df['mileage']/df['auto_age']
df['rarity'] = df.productionDate.apply(lambda x: 1 if x < 1960 else 0)
df['older_10y'] = df.productionDate.apply(lambda x: 1 if x < 2021 - 10 else 0)
df['top2_bodyType'] = df.bodyType.apply(lambda x: 1 if x in ['внедорожник', 'седан'] else 0)
df['rare_bodyType'] = df.bodyType.apply(lambda x: 1 if x in ['микровэн', 'седан-хардтоп', 'лимузин', 'тарга', 'фастбек'] else 0)
df['top5_colors'] = df.color.apply(lambda x: 1 if x in ['черный', 'белый', 'серый', 'синий', 'серебристый'] else 0)
df['rare_colors'] = df.color.apply(lambda x: 1 if x in ['фиолетовый', 'пурпурный', 'золотистый', 'оранжевый', 'жёлтый', 'розовый'] else 0)
df['top2_door_numb'] = df.numberOfDoors.apply(lambda x: 1 if x in [4, 5] else 0)

df.drop(['description', 'productionDate'], axis=1, inplace = True)



In [70]:
bin_cols = ['ПТС', 'Руль']

cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'model_name', 'numberOfDoors', 'vehicleTransmission'
            , 'Привод']

num_cols = ['mileage', 'Владельцы', 'enginePower', 'descr_words_count', 'descr_words_count', 'auto_age']

# избавимся от буквенных значений в бинарных признакаx:
label_encoder = LabelEncoder()

for col in bin_cols:
    df[col] = label_encoder.fit_transform(df[col])
    
for col in cat_cols:
    df[col] = df[col].astype('category').cat.codes
    
# Применим логарифмирование ко всем числовым признакам
for col in num_cols:
    df[col] = df[col].apply(lambda w: np.log(w+1))

In [ ]:
'''
Попробовал преобразовать описания в вектора но до конца не довел
#преобразование description 
#раскладываем каждое описание авто на слова
analyzer = CountVectorizer(stop_words='english').build_analyzer()

descriptions = []
for description in df.description:
    descriptions.append(analyzer(description))
    
# загружаем стоп слова, чтобы избавиться от шума в данных
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords = stopwords.words("russian")

#удаляем стоп слова из описаний авто
for description in descriptions:
    for word in description:
        if word in stopwords:
            description.remove(word)
            
#убираем окончания
snowball = SnowballStemmer(language="russian")

for description in descriptions:
    for i in range(0, len(description)):
        description[i] = snowball.stem(description[i])

model = Word2Vec(descriptions, min_count=40, vector_size=50)

def doc_vectorizer(doc, model):
    doc_vector = []
    num_words = 0
    for word in doc:
        try:
            if num_words == 0:
                doc_vector = model.wv[word]
            else:
                doc_vector = np.add(doc_vector, model[word])
            num_words += 1
        except:
            pass
     
    return np.asarray(doc_vector) / num_words

# Составляем эмбеддинги для наших документов
descriptions_vect = []

for description in descriptions:
    descriptions_vect.append(doc_vectorizer(description, model))
    
df['description'] = descriptions_vect
descriptions_vect = np.array(descriptions_vect)

# обнаруживаем странные значения текстов, заполняем их вектора нулями
for i in range(len(descriptions_vect)):
    if len(descriptions_vect[i])==0:
        descriptions_vect[i] = np.array([0,0,0,0,0,0,0,0,0,0,
                                        0,0,0,0,0,0,0,0,0,0,
                                        0,0,0,0,0,0,0,0,0,0,
                                        0,0,0,0,0,0,0,0,0,0,
                                        0,0,0,0,0,0,0,0,0,0])
        
        
for i in range(10):
        df['vec'+str(i)] = 0.0
        
for df_index in tqdm(range(len(descriptions_vect))):
    for vect_index in range(10):
        df.loc[df_index, 'vec'+str(vect_index)] = descriptions_vect[df_index][vect_index]
        
# t-SNE – метод понижения размерности
from sklearn.manifold import TSNE

# Создаём объект для выполнения t-SNE
tsne = TSNE(n_components=2, random_state=42)

# Преобразуем наши данные, понизив размерность с 50 до 2
descriptions_vect = tsne.fit_transform(descriptions_vect)

In [54]:
# разбиваем на трейн и тест
y = df.query('sample == 1').price

X = df[df['sample']==1]
X = X.drop(['price', 'sample'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=42)

In [56]:
model1 = CatBoostRegressor(iterations = 5000,
                           random_seed = 42,
                           eval_metric='MAPE',
                           custom_metric=['R2', 'MAE'],
                           silent=True)
 
model1.fit(X_train, np.log(y_train))

predict = np.exp(model1.predict(X_test) )
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 12.93%


/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)


In [55]:
model2 = RandomForestRegressor(random_state=42)
 
model2.fit(X_train,np.log(y_train))

predict = np.exp(model2.predict(X_test) )
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 12.76%


/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)


In [57]:
model3 = ExtraTreesRegressor(random_state=42)
 
model3.fit(X_train,np.log(y_train))

predict = np.exp(model3.predict(X_test) )
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Точность модели по метрике MAPE: 12.85%


/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)


In [52]:
model4 = XGBRegressor()

model4.fit(X_train,np.log(y_train))

predict = np.exp(model4.predict(X_test) )
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Точность модели по метрике MAPE: 13.76%


/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)


In [53]:
# делаем стекинг трех лучших моделей, эксперименты показали, что нужно взять именно эти три модели.
meta_model = LinearRegression()

estimators = (
              ('RandForRegr',RandomForestRegressor(random_state=42)),
              ('ExtrTreeRegr',ExtraTreesRegressor(random_state=42)),
              ('CatBoost',CatBoostRegressor())
)

model_final = StackingRegressor(estimators=estimators, final_estimator=meta_model)

model_final.fit(X_train,np.log(y_train))

predict = np.exp(model_final.predict(X_test) )
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Learning rate set to 0.079855
0:	learn: 0.9877058	total: 12.1ms	remaining: 12.1s
1:	learn: 0.9248898	total: 16.4ms	remaining: 8.17s
2:	learn: 0.8680936	total: 19.9ms	remaining: 6.63s
3:	learn: 0.8161091	total: 24ms	remaining: 5.99s
4:	learn: 0.7685689	total: 27.5ms	remaining: 5.47s
5:	learn: 0.7262381	total: 32.8ms	remaining: 5.43s
6:	learn: 0.6866536	total: 37.4ms	remaining: 5.3s
7:	learn: 0.6508427	total: 42ms	remaining: 5.21s
8:	learn: 0.6174354	total: 45.8ms	remaining: 5.04s
9:	learn: 0.5878783	total: 50ms	remaining: 4.95s
10:	learn: 0.5607300	total: 54.2ms	remaining: 4.88s
11:	learn: 0.5356011	total: 58.6ms	remaining: 4.82s
12:	learn: 0.5135156	total: 62.9ms	remaining: 4.78s
13:	learn: 0.4928923	total: 66.9ms	remaining: 4.71s
14:	learn: 0.4746172	total: 71ms	remaining: 4.66s
15:	learn: 0.4575549	total: 75.3ms	remaining: 4.63s
16:	learn: 0.4425913	total: 79.7ms	remaining: 4.61s
17:	learn: 0.4288745	total: 83.9ms	remaining: 4.58s
18:	learn: 0.4164084	total: 88.3ms	remaining: 4.56s
1

166:	learn: 0.2309563	total: 772ms	remaining: 3.85s
167:	learn: 0.2305530	total: 776ms	remaining: 3.84s
168:	learn: 0.2304440	total: 780ms	remaining: 3.84s
169:	learn: 0.2302146	total: 785ms	remaining: 3.83s
170:	learn: 0.2300180	total: 788ms	remaining: 3.82s
171:	learn: 0.2297238	total: 793ms	remaining: 3.81s
172:	learn: 0.2295672	total: 797ms	remaining: 3.81s
173:	learn: 0.2294211	total: 800ms	remaining: 3.8s
174:	learn: 0.2292638	total: 804ms	remaining: 3.79s
175:	learn: 0.2289825	total: 809ms	remaining: 3.79s
176:	learn: 0.2286094	total: 813ms	remaining: 3.78s
177:	learn: 0.2284385	total: 817ms	remaining: 3.77s
178:	learn: 0.2279131	total: 821ms	remaining: 3.77s
179:	learn: 0.2278414	total: 825ms	remaining: 3.76s
180:	learn: 0.2276685	total: 829ms	remaining: 3.75s
181:	learn: 0.2275223	total: 833ms	remaining: 3.75s
182:	learn: 0.2273380	total: 837ms	remaining: 3.74s
183:	learn: 0.2272206	total: 842ms	remaining: 3.73s
184:	learn: 0.2269613	total: 846ms	remaining: 3.73s
185:	learn: 0

339:	learn: 0.2063764	total: 1.56s	remaining: 3.04s
340:	learn: 0.2062435	total: 1.57s	remaining: 3.03s
341:	learn: 0.2061358	total: 1.57s	remaining: 3.02s
342:	learn: 0.2060857	total: 1.58s	remaining: 3.02s
343:	learn: 0.2059796	total: 1.58s	remaining: 3.01s
344:	learn: 0.2058462	total: 1.58s	remaining: 3.01s
345:	learn: 0.2057702	total: 1.59s	remaining: 3s
346:	learn: 0.2056830	total: 1.59s	remaining: 3s
347:	learn: 0.2055861	total: 1.6s	remaining: 2.99s
348:	learn: 0.2055654	total: 1.6s	remaining: 2.98s
349:	learn: 0.2055153	total: 1.6s	remaining: 2.98s
350:	learn: 0.2054377	total: 1.61s	remaining: 2.97s
351:	learn: 0.2052867	total: 1.61s	remaining: 2.97s
352:	learn: 0.2052458	total: 1.62s	remaining: 2.96s
353:	learn: 0.2051834	total: 1.62s	remaining: 2.96s
354:	learn: 0.2050790	total: 1.63s	remaining: 2.95s
355:	learn: 0.2049950	total: 1.63s	remaining: 2.95s
356:	learn: 0.2049478	total: 1.63s	remaining: 2.94s
357:	learn: 0.2049374	total: 1.64s	remaining: 2.94s
358:	learn: 0.2048553

529:	learn: 0.1929756	total: 2.35s	remaining: 2.09s
530:	learn: 0.1929037	total: 2.36s	remaining: 2.08s
531:	learn: 0.1928552	total: 2.36s	remaining: 2.08s
532:	learn: 0.1927950	total: 2.37s	remaining: 2.07s
533:	learn: 0.1927361	total: 2.37s	remaining: 2.07s
534:	learn: 0.1926941	total: 2.37s	remaining: 2.06s
535:	learn: 0.1926320	total: 2.38s	remaining: 2.06s
536:	learn: 0.1925733	total: 2.38s	remaining: 2.05s
537:	learn: 0.1925560	total: 2.38s	remaining: 2.05s
538:	learn: 0.1925145	total: 2.39s	remaining: 2.04s
539:	learn: 0.1924676	total: 2.39s	remaining: 2.04s
540:	learn: 0.1924103	total: 2.4s	remaining: 2.03s
541:	learn: 0.1923754	total: 2.4s	remaining: 2.03s
542:	learn: 0.1923170	total: 2.4s	remaining: 2.02s
543:	learn: 0.1922556	total: 2.41s	remaining: 2.02s
544:	learn: 0.1921398	total: 2.41s	remaining: 2.01s
545:	learn: 0.1920967	total: 2.42s	remaining: 2.01s
546:	learn: 0.1920415	total: 2.42s	remaining: 2s
547:	learn: 0.1919949	total: 2.42s	remaining: 2s
548:	learn: 0.1919417

720:	learn: 0.1841501	total: 3.14s	remaining: 1.22s
721:	learn: 0.1840811	total: 3.15s	remaining: 1.21s
722:	learn: 0.1840369	total: 3.15s	remaining: 1.21s
723:	learn: 0.1839979	total: 3.16s	remaining: 1.2s
724:	learn: 0.1839608	total: 3.16s	remaining: 1.2s
725:	learn: 0.1839170	total: 3.16s	remaining: 1.19s
726:	learn: 0.1838872	total: 3.17s	remaining: 1.19s
727:	learn: 0.1838466	total: 3.17s	remaining: 1.19s
728:	learn: 0.1837881	total: 3.18s	remaining: 1.18s
729:	learn: 0.1837357	total: 3.18s	remaining: 1.18s
730:	learn: 0.1836716	total: 3.19s	remaining: 1.17s
731:	learn: 0.1836375	total: 3.19s	remaining: 1.17s
732:	learn: 0.1835972	total: 3.19s	remaining: 1.16s
733:	learn: 0.1835684	total: 3.2s	remaining: 1.16s
734:	learn: 0.1835230	total: 3.2s	remaining: 1.15s
735:	learn: 0.1834893	total: 3.21s	remaining: 1.15s
736:	learn: 0.1834376	total: 3.21s	remaining: 1.15s
737:	learn: 0.1833893	total: 3.21s	remaining: 1.14s
738:	learn: 0.1833662	total: 3.22s	remaining: 1.14s
739:	learn: 0.18

...

274:	learn: 0.2137236	total: 1.05s	remaining: 2.77s
275:	learn: 0.2135475	total: 1.06s	remaining: 2.77s
276:	learn: 0.2134332	total: 1.06s	remaining: 2.77s
277:	learn: 0.2132990	total: 1.06s	remaining: 2.76s
278:	learn: 0.2131622	total: 1.07s	remaining: 2.76s
279:	learn: 0.2130762	total: 1.07s	remaining: 2.76s
280:	learn: 0.2128926	total: 1.07s	remaining: 2.75s
281:	learn: 0.2127943	total: 1.08s	remaining: 2.75s
282:	learn: 0.2126286	total: 1.08s	remaining: 2.75s
283:	learn: 0.2124690	total: 1.09s	remaining: 2.74s
284:	learn: 0.2122878	total: 1.09s	remaining: 2.74s
285:	learn: 0.2121848	total: 1.09s	remaining: 2.73s
286:	learn: 0.2120568	total: 1.1s	remaining: 2.73s
287:	learn: 0.2118989	total: 1.1s	remaining: 2.73s
288:	learn: 0.2118042	total: 1.11s	remaining: 2.72s
289:	learn: 0.2116717	total: 1.11s	remaining: 2.72s
290:	learn: 0.2115949	total: 1.11s	remaining: 2.72s
291:	learn: 0.2114819	total: 1.12s	remaining: 2.71s
292:	learn: 0.2112330	total: 1.12s	remaining: 2.71s
293:	learn: 0.

480:	learn: 0.1955234	total: 1.84s	remaining: 1.98s
481:	learn: 0.1954646	total: 1.84s	remaining: 1.98s
482:	learn: 0.1953868	total: 1.85s	remaining: 1.98s
483:	learn: 0.1953314	total: 1.85s	remaining: 1.97s
484:	learn: 0.1952537	total: 1.85s	remaining: 1.97s
485:	learn: 0.1952069	total: 1.86s	remaining: 1.97s
486:	learn: 0.1951348	total: 1.86s	remaining: 1.96s
487:	learn: 0.1950432	total: 1.86s	remaining: 1.96s
488:	learn: 0.1949298	total: 1.87s	remaining: 1.95s
489:	learn: 0.1948642	total: 1.87s	remaining: 1.95s
490:	learn: 0.1947924	total: 1.88s	remaining: 1.95s
491:	learn: 0.1947283	total: 1.88s	remaining: 1.94s
492:	learn: 0.1946841	total: 1.88s	remaining: 1.94s
493:	learn: 0.1946346	total: 1.89s	remaining: 1.93s
494:	learn: 0.1945899	total: 1.89s	remaining: 1.93s
495:	learn: 0.1945246	total: 1.9s	remaining: 1.93s
496:	learn: 0.1944681	total: 1.9s	remaining: 1.92s
497:	learn: 0.1944198	total: 1.9s	remaining: 1.92s
498:	learn: 0.1942977	total: 1.91s	remaining: 1.92s
499:	learn: 0.1

686:	learn: 0.1845607	total: 2.63s	remaining: 1.2s
687:	learn: 0.1845175	total: 2.63s	remaining: 1.19s
688:	learn: 0.1844828	total: 2.63s	remaining: 1.19s
689:	learn: 0.1844559	total: 2.64s	remaining: 1.19s
690:	learn: 0.1844202	total: 2.64s	remaining: 1.18s
691:	learn: 0.1843756	total: 2.65s	remaining: 1.18s
692:	learn: 0.1843370	total: 2.65s	remaining: 1.17s
693:	learn: 0.1842779	total: 2.65s	remaining: 1.17s
694:	learn: 0.1842419	total: 2.66s	remaining: 1.17s
695:	learn: 0.1841984	total: 2.66s	remaining: 1.16s
696:	learn: 0.1841215	total: 2.67s	remaining: 1.16s
697:	learn: 0.1840599	total: 2.67s	remaining: 1.15s
698:	learn: 0.1840128	total: 2.67s	remaining: 1.15s
699:	learn: 0.1839802	total: 2.68s	remaining: 1.15s
700:	learn: 0.1839409	total: 2.68s	remaining: 1.14s
701:	learn: 0.1838681	total: 2.68s	remaining: 1.14s
702:	learn: 0.1838244	total: 2.69s	remaining: 1.14s
703:	learn: 0.1837835	total: 2.69s	remaining: 1.13s
704:	learn: 0.1837627	total: 2.69s	remaining: 1.13s
705:	learn: 0

893:	learn: 0.1767939	total: 3.41s	remaining: 405ms
894:	learn: 0.1767531	total: 3.42s	remaining: 401ms
895:	learn: 0.1767232	total: 3.42s	remaining: 397ms
896:	learn: 0.1766733	total: 3.42s	remaining: 393ms
897:	learn: 0.1766473	total: 3.43s	remaining: 390ms
898:	learn: 0.1766068	total: 3.43s	remaining: 386ms
899:	learn: 0.1765724	total: 3.44s	remaining: 382ms
900:	learn: 0.1765439	total: 3.44s	remaining: 378ms
901:	learn: 0.1765115	total: 3.44s	remaining: 374ms
902:	learn: 0.1764572	total: 3.45s	remaining: 370ms
903:	learn: 0.1764292	total: 3.45s	remaining: 367ms
904:	learn: 0.1764012	total: 3.46s	remaining: 363ms
905:	learn: 0.1763614	total: 3.46s	remaining: 359ms
906:	learn: 0.1763353	total: 3.46s	remaining: 355ms
907:	learn: 0.1762805	total: 3.47s	remaining: 352ms
908:	learn: 0.1762651	total: 3.47s	remaining: 348ms
909:	learn: 0.1762386	total: 3.48s	remaining: 344ms
910:	learn: 0.1761908	total: 3.48s	remaining: 340ms
911:	learn: 0.1761633	total: 3.48s	remaining: 336ms
912:	learn: 

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)


In [ ]:
#так и не отработало до конца
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 25)]
max_features = ['log2', 'sqrt', 'auto']
max_depth = [int(x) for x in np.linspace(start = 1, stop = 200, num = 20)]
min_samples_split = [int(x) for x in np.linspace(start = 2, stop = 10, num = 1)]
min_samples_leaf = [int(x) for x in np.linspace(start = 1, stop = 10, num = 1)]
bootstrap = [True, False]
param_dist = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
rs = RandomizedSearchCV(model2, 
                        param_dist, 
                        n_iter = 100, 
                        cv = 3, 
                        verbose = 1, 
                        n_jobs=-1, 
                        random_state=42)
rs.fit(X_train, np.log(y_train))
rs.best_params_

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is depre

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is depre

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is depre

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is depre

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is depre

In [34]:
'''Еле-еле отработало до конца только на первой 1000 значений, ноутбук чуть не умер, в итоге вообще
не помогло, качество только упало.'''
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [80, 100, 150, 200],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'min_samples_split': [2, 3, 4, 5, 10],
    'n_estimators': [100, 200, 300, 400, 1000]
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = model2, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

grid_search.fit(X[:1000], np.log(y[:1000]))
grid_search.best_params_

Fitting 3 folds for each of 1000 candidates, totalling 3000 fits
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   3.3s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=4, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=4, n_estimators=400; total time=   1.1s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=4, n_estimators=1000; total time=   2.8s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.8s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=   2.4s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=2, min_samples_split=2, n_estimators=1000; total time= 

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is depre

[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.7s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   3.1s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=4, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=4, n_estimators=400; total time=   1.2s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=5, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=5, n_estimators=400; total time=   1.2s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is depre

[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.9s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=2, n_estimators=1000; total time=   3.2s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=4, n_estimators=300; total time=   0.9s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=4, n_estimators=1000; total time=   2.8s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.2s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.3s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.7s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   1.0s
[CV] END bootstrap=True, max_depth=80, min_samples_leaf=2,

...

[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=4, n_estimators=100; total time=   0.4s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=4, n_estimators=300; total time=   1.2s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=4, n_estimators=1000; total time=   3.9s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.3s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.4s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.7s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   1.4s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=10, n_estimators=1000; total time=   3.3s
[CV] END bootstrap=False, max_depth=200, 

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is depre

[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=4, n_estimators=300; total time=   1.2s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=4, n_estimators=1000; total time=   4.2s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   1.0s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=1, min_samples_split=10, n_estimators=400; total time=   1.4s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.4s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=   1.1s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=2, min_samples_split=2, n_estimators=400; total time=   1.5s
[CV] END bootstrap=False, max_depth=200, min_samples_leaf=2, min_samples_split=3, n_estimators=200; total time=   0.8s
[CV] END bootstrap=False, max_depth=200, min_

/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  y_true = check_array(y_true, ensure_2d=False, dtype=dtype)
/Users/kirill/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_regression.py:95: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is depre

{'bootstrap': False,
 'max_depth': 80,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'n_estimators': 100}

In [80]:
#записываем submission
submission = pd.read_csv('/Users/kirill/Downloads/sf-dst-car-price-prediction/sample_submission.csv')
submission = submission.set_index('sell_id')

In [73]:
X_test = df[df['sample']==0]
X_test = X_test.drop(['price', 'sample'], axis = 1)

y_predict = np.exp(model_final.predict(X_test) )

submission.price = y_predict

In [83]:
submission.to_csv('sum_final')